# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 20 2022 12:00AM,237693,10,Stone8041619,Acute Outpatient Solutions,Released
1,May 20 2022 12:00AM,237694,10,6482857,Acute Outpatient Solutions,Released
2,May 20 2022 12:00AM,237696,10,8041612,"Herbivore Botanicals, LLC.",Released
3,May 20 2022 12:00AM,237696,10,8041636,"Herbivore Botanicals, LLC.",Released
4,May 20 2022 12:00AM,237696,10,8041640,"Herbivore Botanicals, LLC.",Released
5,May 20 2022 12:00AM,237702,10,8041624-F,"E5 Pharma, LLC",Released
6,May 20 2022 12:00AM,237702,10,8041625-F,"E5 Pharma, LLC",Released
7,May 20 2022 12:00AM,237702,10,8041626-F,"E5 Pharma, LLC",Released
8,May 20 2022 12:00AM,237702,10,8041627-F,"E5 Pharma, LLC",Released
9,May 20 2022 12:00AM,237702,10,8041628-F,"E5 Pharma, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,237698,Released,5
26,237700,Released,1
27,237702,Released,12
28,237703,Released,1
29,237704,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237698,NaN,NaN,5.0
237700,NaN,NaN,1.0
237702,NaN,NaN,12.0
237703,NaN,NaN,1.0
237704,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,3.0,0.0,2.0
237597,0.0,0.0,1.0
237598,0.0,0.0,1.0
237608,19.0,3.0,0.0
237610,3.0,5.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,3,0,2
237597,0,0,1
237598,0,0,1
237608,19,3,0
237610,3,5,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,3,0,2
1,237597,0,0,1
2,237598,0,0,1
3,237608,19,3,0
4,237610,3,5,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,3,,2
1,237597,,,1
2,237598,,,1
3,237608,19,3,
4,237610,3,5,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 20 2022 12:00AM,237693,10,Acute Outpatient Solutions
1,May 20 2022 12:00AM,237694,10,Acute Outpatient Solutions
2,May 20 2022 12:00AM,237696,10,"Herbivore Botanicals, LLC."
5,May 20 2022 12:00AM,237702,10,"E5 Pharma, LLC"
17,May 20 2022 12:00AM,237704,10,"Herbivore Botanicals, LLC."
18,May 19 2022 5:23PM,237703,21,"NBTY Global, Inc."
19,May 19 2022 4:41PM,237700,18,Hush Hush
20,May 19 2022 4:30PM,237698,19,ACG North America LLC
25,May 19 2022 3:35PM,237690,19,"Emersa Waterbox, LLC"
26,May 19 2022 3:34PM,237689,16,"Goodwin Biotechnology, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 20 2022 12:00AM,237693,10,Acute Outpatient Solutions,,,1
1,May 20 2022 12:00AM,237694,10,Acute Outpatient Solutions,,,1
2,May 20 2022 12:00AM,237696,10,"Herbivore Botanicals, LLC.",,,3
3,May 20 2022 12:00AM,237702,10,"E5 Pharma, LLC",,,12
4,May 20 2022 12:00AM,237704,10,"Herbivore Botanicals, LLC.",,,1
5,May 19 2022 5:23PM,237703,21,"NBTY Global, Inc.",,,1
6,May 19 2022 4:41PM,237700,18,Hush Hush,,,1
7,May 19 2022 4:30PM,237698,19,ACG North America LLC,,,5
8,May 19 2022 3:35PM,237690,19,"Emersa Waterbox, LLC",,,1
9,May 19 2022 3:34PM,237689,16,"Goodwin Biotechnology, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 12:00AM,237693,10,Acute Outpatient Solutions,1,,
1,May 20 2022 12:00AM,237694,10,Acute Outpatient Solutions,1,,
2,May 20 2022 12:00AM,237696,10,"Herbivore Botanicals, LLC.",3,,
3,May 20 2022 12:00AM,237702,10,"E5 Pharma, LLC",12,,
4,May 20 2022 12:00AM,237704,10,"Herbivore Botanicals, LLC.",1,,
5,May 19 2022 5:23PM,237703,21,"NBTY Global, Inc.",1,,
6,May 19 2022 4:41PM,237700,18,Hush Hush,1,,
7,May 19 2022 4:30PM,237698,19,ACG North America LLC,5,,
8,May 19 2022 3:35PM,237690,19,"Emersa Waterbox, LLC",1,,
9,May 19 2022 3:34PM,237689,16,"Goodwin Biotechnology, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 12:00AM,237693,10,Acute Outpatient Solutions,1,,
1,May 20 2022 12:00AM,237694,10,Acute Outpatient Solutions,1,,
2,May 20 2022 12:00AM,237696,10,"Herbivore Botanicals, LLC.",3,,
3,May 20 2022 12:00AM,237702,10,"E5 Pharma, LLC",12,,
4,May 20 2022 12:00AM,237704,10,"Herbivore Botanicals, LLC.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 12:00AM,237693,10,Acute Outpatient Solutions,1.0,NaN,NaN
1,May 20 2022 12:00AM,237694,10,Acute Outpatient Solutions,1.0,NaN,NaN
2,May 20 2022 12:00AM,237696,10,"Herbivore Botanicals, LLC.",3.0,NaN,NaN
3,May 20 2022 12:00AM,237702,10,"E5 Pharma, LLC",12.0,NaN,NaN
4,May 20 2022 12:00AM,237704,10,"Herbivore Botanicals, LLC.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 12:00AM,237693,10,Acute Outpatient Solutions,1.0,0.0,0.0
1,May 20 2022 12:00AM,237694,10,Acute Outpatient Solutions,1.0,0.0,0.0
2,May 20 2022 12:00AM,237696,10,"Herbivore Botanicals, LLC.",3.0,0.0,0.0
3,May 20 2022 12:00AM,237702,10,"E5 Pharma, LLC",12.0,0.0,0.0
4,May 20 2022 12:00AM,237704,10,"Herbivore Botanicals, LLC.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2139020,34.0,12.0,22.0
16,475350,2.0,0.0,0.0
17,237654,0.0,1.0,0.0
18,950538,4.0,0.0,0.0
19,713076,7.0,0.0,0.0
20,712874,22.0,0.0,3.0
21,237703,1.0,0.0,0.0
22,475281,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2139020,34.0,12.0,22.0
1,16,475350,2.0,0.0,0.0
2,17,237654,0.0,1.0,0.0
3,18,950538,4.0,0.0,0.0
4,19,713076,7.0,0.0,0.0
5,20,712874,22.0,0.0,3.0
6,21,237703,1.0,0.0,0.0
7,22,475281,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,34.0,12.0,22.0
1,16,2.0,0.0,0.0
2,17,0.0,1.0,0.0
3,18,4.0,0.0,0.0
4,19,7.0,0.0,0.0
5,20,22.0,0.0,3.0
6,21,1.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,34.0
1,16,Released,2.0
2,17,Released,0.0
3,18,Released,4.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,17,18,19,20,21,22
Status,,,,,,,,
Completed,22.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
Executing,12.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Released,34.0,2.0,0.0,4.0,7.0,22.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,17,18,19,20,21,22
0,Completed,22.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,Executing,12.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Released,34.0,2.0,0.0,4.0,7.0,22.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,17,18,19,20,21,22
0,Completed,22.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,Executing,12.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Released,34.0,2.0,0.0,4.0,7.0,22.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()